In [2]:
!pip install scikit-learn pandas numpy joblib matplotlib

import re, random, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import make_pipeline
import joblib

# ----------------------------
# 1) Dataset sintético realista
# ----------------------------
random.seed(42); np.random.seed(42) # Explicacion detallada: Esto asegura que los resultados sean reproducibles al fijar la semilla para las funciones aleatorias de Python y NumPy.

positivos = [
    "Excelente servicio","Muy buena atención","Me encantó el producto",
    "Rápido y confiable","Todo llegó perfecto","Calidad superior",
    "Lo recomiendo totalmente","Volveré a comprar","Precio justo y buena calidad",
    "El soporte fue amable","Experiencia increíble","Funcionó mejor de lo esperado",
    "Entregado a tiempo","Muy satisfecho","Cinco estrellas",
    "La comida estaba deliciosa","El empaque impecable","Súper recomendable",
    "Buen trato del personal","Gran experiencia"
]

negativos = [
    "Pésimo servicio","Muy mala atención","Odio este producto",
    "Lento y poco confiable","Llegó dañado","Calidad terrible",
    "No lo recomiendo","No vuelvo a comprar","Caro y mala calidad",
    "El soporte fue grosero","Experiencia horrible","Peor de lo esperado",
    "Entregado tarde","Muy decepcionado","Una estrella",
    "La comida estaba fría","El empaque roto","Nada recomendable",
    "Mal trato del personal","Mala experiencia"
]

def variantes(frase):
    extras = ["", "!", "!!", " 🙂", " 😡", " de verdad", " en serio", " 10/10", " 1/10",
              " súper", " la verdad", " jamás", " nunca", " para nada"]
    return frase + random.choice(extras)

pos = [variantes(p) for _ in range(8) for p in positivos] # Aumentar datos con variantes
neg = [variantes(n) for _ in range(8) for n in negativos] # Aumentar datos con variantes
textos = pos + neg # Mezclar positivos y negativos
etiquetas = [1]*len(pos) + [0]*len(neg) # 1=Positivo, 0=Negativo

df = pd.DataFrame({"texto": textos, "etiqueta": etiquetas}).sample(frac=1, random_state=42).reset_index(drop=True)
# Mezclar filas aleatoriamente

print("Muestras:", df.shape[0], " | Positivos:", df.etiqueta.sum(), " | Negativos:", len(df)-df.etiqueta.sum())

Muestras: 320  | Positivos: 160  | Negativos: 160
Muestras: 320  | Positivos: 160  | Negativos: 160


In [3]:
# ----------------------------
# 2) Limpieza simple
# ----------------------------
def limpiar(s: str) -> str:
    s = s.lower()
    s = re.sub(r"[^a-záéíóúñü0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["texto_clean"] = df["texto"].apply(limpiar)

In [4]:
# ----------------------------
# 3) Split estratificado + baseline
# ----------------------------
X_train_text, X_test_text, y_train, y_test = train_test_split(
    df["texto_clean"], df["etiqueta"], test_size=0.2, random_state=42, stratify=df["etiqueta"]
) # Mantener proporción de clases en train/test

# Baseline: predecir siempre la clase mayoritaria (en este caso, positivo=1 o negativo=0)
mayoritaria = int(round(y_train.mean()))  # 0 o 1
baseline = (y_test == mayoritaria).mean() # Proporción de la clase mayoritaria en test
print(f"Baseline (clase mayoritaria): {baseline:.3f}") # Ej: 0.505 si clases balanceadas

Baseline (clase mayoritaria): 0.500


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=30000, ngram_range=(1,2), min_df=2)
X_train = vectorizer.fit_transform(X_train_text)  # aprende vocabulario y transforma train
X_test  = vectorizer.transform(X_test_text)       # solo transforma test (no fit)

In [6]:
from sklearn.svm import LinearSVC

clf = LinearSVC(class_weight="balanced", random_state=42)
clf.fit(X_train, y_train)

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,verbose,0
,random_state,42


In [7]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd

pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f"\nAccuracy en test: {acc:.3f}  |  Mejora vs baseline: {acc - baseline:.3f}\n")

print("Reporte por clase:")
print(classification_report(y_test, pred, digits=3))

cm = confusion_matrix(y_test, pred, labels=[0,1])
print("\nMatriz de confusión:")
print(pd.DataFrame(cm, index=["Real 0 (neg)", "Real 1 (pos)"], columns=["Pred 0 (neg)", "Pred 1 (pos)"]))


Accuracy en test: 1.000  |  Mejora vs baseline: 0.500

Reporte por clase:
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        32
           1      1.000     1.000     1.000        32

    accuracy                          1.000        64
   macro avg      1.000     1.000     1.000        64
weighted avg      1.000     1.000     1.000        64


Matriz de confusión:
              Pred 0 (neg)  Pred 1 (pos)
Real 0 (neg)            32             0
Real 1 (pos)             0            32


In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    TfidfVectorizer(max_features=30000, ngram_range=(1,2), min_df=2),
    LinearSVC(class_weight="balanced", random_state=42)
)
scores = cross_val_score(pipe, df["texto_clean"], df["etiqueta"], cv=5, scoring="f1_macro")
print(f"\nCV (5-fold) F1_macro: media={scores.mean():.3f}  ±{scores.std():.3f}")


CV (5-fold) F1_macro: media=1.000  ±0.000


In [9]:
def predecir(textos_nuevos):
    tx = [limpiar(t) for t in textos_nuevos]
    Xn = vectorizer.transform(tx)
    p = clf.predict(Xn)
    return ["positivo" if i==1 else "negativo" for i in p]

nuevos = [
    "El envío fue rapidísimo y el empaque llegó impecable, gracias!",
    "Demoraron demasiado y además nadie respondió los mensajes",
    "Calidad/precio brutal, quedé muy satisfecho",
    "No lo recomiendo, salió defectuoso y me tocó devolverlo"
]
print("\nPredicciones en textos nuevos:")
for t, etiqueta in zip(nuevos, predecir(nuevos)):
    print(f"- {t}  ->  {etiqueta}")


Predicciones en textos nuevos:
- El envío fue rapidísimo y el empaque llegó impecable, gracias!  ->  positivo
- Demoraron demasiado y además nadie respondió los mensajes  ->  negativo
- Calidad/precio brutal, quedé muy satisfecho  ->  positivo
- No lo recomiendo, salió defectuoso y me tocó devolverlo  ->  negativo


In [10]:
import joblib

joblib.dump(vectorizer, "tfidf.joblib")
joblib.dump(clf, "modelo.joblib")
print("\nModelo y vectorizador guardados.")

vec = joblib.load("tfidf.joblib")
model = joblib.load("modelo.joblib")
Xn = vec.transform(["La compra fue excelente, todo perfecto"])
print("Pred loaded model:", "positivo" if model.predict(Xn)[0]==1 else "negativo")


Modelo y vectorizador guardados.
Pred loaded model: positivo


In [11]:
# ----------------------------
# DEMOSTRACIÓN: Más ejemplos de clasificación
# ----------------------------

ejemplos_demo = [
    # === POSITIVOS CLAROS ===
    "¡Excelente compra! El producto llegó en perfectas condiciones y el servicio fue impecable.",
    "Estoy muy satisfecho, superó mis expectativas. Definitivamente vuelvo a comprar.",
    "La calidad es increíble y el precio muy justo. Altamente recomendado.",
    "Entrega rápida, empaque impecable, producto de primera. 5 estrellas.",
    
    # === NEGATIVOS CLAROS ===
    "Pésimo servicio. El producto llegó dañado y nadie respondió mis reclamos.",
    "Horrible experiencia. Nunca más compro aquí. Total desperdicio de dinero.",
    "Muy decepcionado. El producto no funciona como prometieron en la descripción.",
    "El peor pedido de mi vida. Llegó roto, atención al cliente grosera. Una estrella.",
    
    # === MIXTOS / AMBIGUOS ===
    "El producto está bien pero tardó demasiado en llegar. Regular.",
    "Tiene buena calidad pero es más caro de lo esperado.",
    "Llegó rápido pero la calidad dejó que desear.",
    "El servicio fue lento pero al final quedé satisfecho.",
]

predicciones_demo = predecir(ejemplos_demo)

print("\n" + "="*80)
print("DEMOSTRACIÓN: Clasificación de comentarios con el modelo entrenado")
print("="*80 + "\n")

for i, (texto, etiqueta) in enumerate(zip(ejemplos_demo, predicciones_demo), 1):
    emoji = "✅" if etiqueta == "positivo" else "❌"
    print(f"{i:2d}. [{emoji}] {etiqueta.upper():<10} | {texto}")
    print()

print("="*80)
print(f"Total ejemplos: {len(ejemplos_demo)}")
print(f"Positivos predichos: {sum(1 for e in predicciones_demo if e == 'positivo')}")
print(f"Negativos predichos: {sum(1 for e in predicciones_demo if e == 'negativo')}")
print("="*80)


DEMOSTRACIÓN: Clasificación de comentarios con el modelo entrenado

 1. [✅] POSITIVO   | ¡Excelente compra! El producto llegó en perfectas condiciones y el servicio fue impecable.

 2. [✅] POSITIVO   | Estoy muy satisfecho, superó mis expectativas. Definitivamente vuelvo a comprar.

 3. [✅] POSITIVO   | La calidad es increíble y el precio muy justo. Altamente recomendado.

 4. [✅] POSITIVO   | Entrega rápida, empaque impecable, producto de primera. 5 estrellas.

 5. [❌] NEGATIVO   | Pésimo servicio. El producto llegó dañado y nadie respondió mis reclamos.

 6. [❌] NEGATIVO   | Horrible experiencia. Nunca más compro aquí. Total desperdicio de dinero.

 7. [❌] NEGATIVO   | Muy decepcionado. El producto no funciona como prometieron en la descripción.

 8. [❌] NEGATIVO   | El peor pedido de mi vida. Llegó roto, atención al cliente grosera. Una estrella.

 9. [✅] POSITIVO   | El producto está bien pero tardó demasiado en llegar. Regular.

10. [✅] POSITIVO   | Tiene buena calidad pero es má